# Preprocesing Seguridad

## 1. Carga de librerias y funciones

In [4]:
import os
import numpy as np
import pandas as pd

In [5]:
def quitar_cero(texto):
    if texto[0] == '0':
        return texto[1:]
    else:
        return texto

## 2. Carga de datos

In [6]:
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
municipios['Id_Municipio_Dane_text'] = municipios['Id_Municipio_Dane'].astype(str)
#municipios['Id_Municipio_Dane_text']=municipios['Id_Municipio_Dane_text']+'000'


In [ ]:
# Leer el archivo CSV como DataFrame
df = pd.read_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_Municipios.CSV', sep=';',encoding='latin-1',dtype={'CODIGO DANE': str} )

In [10]:
df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,AGRUPA EDAD PERSONA,CANTIDAD,Tipo_delito
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,1/01/2010,FEMENINO,ADULTOS,1,Abigeato
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,1/01/2010,MASCULINO,ADULTOS,1,Abigeato
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2/01/2010,FEMENINO,ADULTOS,1,Abigeato
3,CASANARE,PORE,85263000,NO REPORTADO,2/01/2010,MASCULINO,ADULTOS,1,Abigeato
4,CALDAS,NEIRA,17486000,NO REPORTADO,3/01/2010,MASCULINO,ADULTOS,1,Abigeato


## 3. Transformación

In [11]:
#Se agreag la column año
df['Año'] = df['FECHA HECHO'].str[-4:]

In [12]:
df['temp_ARMAS_MEDIOS']=df['ARMAS MEDIOS']
df['temp_CODIGO DANE']=df['CODIGO DANE']

#df.loc[df['Tipo_delito'] == 'hurto_a_comercio', 'temp_ARMAS_MEDIOS'] = 'X'

df.loc[(df['Tipo_delito'] == 'hurto_a_comercio') & (df['Año'] == '2013'), 'temp_ARMAS_MEDIOS'] = df['CODIGO DANE']
df.loc[(df['Tipo_delito'] == 'hurto_a_comercio') & (df['Año'] == '2013'), 'temp_CODIGO DANE'] = df['ARMAS MEDIOS']

In [13]:
df['temp_CODIGO DANE']=df['temp_CODIGO DANE'].astype(str)
df['Cod_dane'] = df['temp_CODIGO DANE'].apply(lambda x: quitar_cero(x))
#df['Cod_dane'] = df['temp_CODIGO DANE'].apply(lambda x: x[:5])
df['Cod_dane'] = df['Cod_dane'].str.slice(stop=-3)



In [14]:
df = pd.merge(df, municipios, left_on='Cod_dane',right_on='Id_Municipio_Dane_text', how='left')

df = df.dropna(subset=['Id_Municipio_Dane'])
#resultado[resultado['Id_Municipio_Dane'].isna()]
df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,AGRUPA EDAD PERSONA,CANTIDAD,Tipo_delito,Año,temp_ARMAS_MEDIOS,temp_CODIGO DANE,Cod_dane,Id_Municipio_Dane,Nombre_Municipio,Id_Departamento_Dane,Nombre_Departamento,Id_Municipio_Dane_text
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,1/01/2010,FEMENINO,ADULTOS,1,Abigeato,2010,NO REPORTADO,41132000,41132,41132.0,Campoalegre,41.0,Huila,41132
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,1/01/2010,MASCULINO,ADULTOS,1,Abigeato,2010,NO REPORTADO,50683000,50683,50683.0,San Juan de Arama,50.0,Meta,50683
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2/01/2010,FEMENINO,ADULTOS,1,Abigeato,2010,NO REPORTADO,15087000,15087,15087.0,Belén,15.0,Boyacá,15087
3,CASANARE,PORE,85263000,NO REPORTADO,2/01/2010,MASCULINO,ADULTOS,1,Abigeato,2010,NO REPORTADO,85263000,85263,85263.0,Pore,85.0,Casanare,85263
4,CALDAS,NEIRA,17486000,NO REPORTADO,3/01/2010,MASCULINO,ADULTOS,1,Abigeato,2010,NO REPORTADO,17486000,17486,17486.0,Neira,17.0,Caldas,17486


# 4. Categoria de delito por año

In [30]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','Tipo_delito','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['Tipo_delito'] + '_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [31]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [32]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [33]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df.csv')

In [36]:
df_pivot.head()

,Cat_seguridad,Seguridad_Abigeato_2010,Seguridad_Abigeato_2011,Seguridad_Abigeato_2012,Seguridad_Abigeato_2013,Seguridad_Abigeato_2014,Seguridad_Abigeato_2015,Seguridad_Abigeato_2016,Seguridad_Abigeato_2017,Seguridad_Abigeato_2018,Seguridad_Abigeato_2019,...,Seguridad_pirateria_terrestre_2013,Seguridad_pirateria_terrestre_2014,Seguridad_pirateria_terrestre_2015,Seguridad_pirateria_terrestre_2016,Seguridad_pirateria_terrestre_2017,Seguridad_pirateria_terrestre_2018,Seguridad_pirateria_terrestre_2019,Seguridad_pirateria_terrestre_2020,Seguridad_pirateria_terrestre_2021,Seguridad_pirateria_terrestre_2022
Id_Municipio_Dane,Nombre_Municipio,,,,,,,,,,,,,,,,,,,,,
5001.0,Medellín,1,3,3,4,4,4,3,10,8,5,...,61,16,6,9,10,2,2,6,1,0
5002.0,Abejorral,0,0,0,0,1,5,1,5,8,1,...,0,0,0,0,1,0,0,0,0,0
5004.0,Abriaquí,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5021.0,Alejandría,0,0,0,1,0,1,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
5030.0,Amagá,0,0,1,0,0,1,0,3,2,0,...,2,0,0,0,0,0,0,0,0,0


# 5. Total delitos por año

In [37]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [38]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [39]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [22]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df_total_año.csv')

In [40]:
df['GENERO'].value_counts()

MASCULINO         2012914
FEMENINO          1517819
NO REPORTA           2556
NO APLICA             542
NO RESPORTADO         372
NO REPORTADO          364
NO REPORTADO           68
-                      63
NO REGISTRA            45
Name: GENERO, dtype: int64

In [41]:
df_pivot.head()

,Cat_seguridad,Seguridad_2010,Seguridad_2011,Seguridad_2012,Seguridad_2013,Seguridad_2014,Seguridad_2015,Seguridad_2016,Seguridad_2017,Seguridad_2018,Seguridad_2019,Seguridad_2020,Seguridad_2021,Seguridad_2022,Seguridad_2023
Id_Municipio_Dane,Nombre_Municipio,,,,,,,,,,,,,,
5001.0,Medellín,7200,12061,15380,16112,20283,24761,34641,41254,48817,53158,37144,44280,51790,656
5002.0,Abejorral,18,13,12,20,39,95,132,135,136,92,90,107,107,4
5004.0,Abriaquí,0,3,0,0,1,3,6,9,2,10,16,3,4,0
5021.0,Alejandría,0,3,1,6,5,34,21,29,21,44,24,28,47,3
5030.0,Amagá,20,34,103,57,102,156,316,326,271,287,230,208,229,11


# 6. Genero

In [24]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','GENERO','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['GENERO'] + '_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [25]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [26]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [27]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df_genero.csv')